In [1]:
import re

import numpy as np
import pandas as pd
import plotly.express as px
from Project.Database import Db
from sklearn.decomposition import PCA
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import MinMaxScaler

In [4]:
scaler = MinMaxScaler()
detector = IsolationForest(n_estimators=1000)
pc = PCA(n_components=0.95, svd_solver="full")

In [5]:
year1_minutes, year1_minutes_meta = Db.load_data(hourly=False, meata=True)
year1_minutes["Timestamp"] = pd.to_datetime(year1_minutes["Timestamp"], format="%Y-%m-%d %H:%M:%S%z")

TypeError: load_data() got an unexpected keyword argument 'meata'

In [ ]:
plugs = year1_minutes_meta.loc[
    lambda self: (self['Parameter'] == "Status_OnOff") & (self["Measurement_Location"] == "Kitchen") & (
        self["Unnamed: 0"].str.contains("Plug"))]["Unnamed: 0"].tolist()

In [ ]:
data_floats = scaler.fit_transform(year1_minutes.loc[:, (year1_minutes.dtypes == "float64")].fillna(0))
non_outliers = np.where(detector.fit_predict(X=data_floats) == 1)

In [ ]:
len(non_outliers[0]) / year1_minutes.shape[0]

In [3]:
kitchen_plugs = year1_minutes.loc[non_outliers[0].tolist(), ["Timestamp", "Load_KPlugLoadsPowerUsage"] + plugs].loc[
    lambda self: ((self[plugs] < 0).sum(1) == 0) & ((self[plugs] > 1).sum(1) == 0)].copy()
# kitchen_plugs = kitchen_plugs
kitchen_plugs["Status"] = kitchen_plugs[plugs].sum(axis=1)

rows = 10000
kitchen_plugs = kitchen_plugs.loc[:rows]

NameError: name 'year1_minutes' is not defined

In [57]:
appliance_dict = {}
for appliance in plugs:
    name = " ".join(re.findall('[A-Z][^A-Z]*', appliance.split("Load")[-1]))
    appliance_switch = kitchen_plugs[(kitchen_plugs[appliance] != kitchen_plugs[appliance].shift(1))][
                           ["Timestamp", appliance]][1:]
    appliance_dict[name] = pd.DataFrame(
        {"Timestamp": [appliance_switch.loc[index, "Timestamp"] for index in appliance_switch.index],
         "Appliance": name,
         "Color": ["green" if appliance_switch.loc[index, appliance] == 1 else "red" for index in
                   appliance_switch.index]})

In [58]:
fig = px.scatter(kitchen_plugs,
                 x=[time.timestamp() * 1000 for time in kitchen_plugs["Timestamp"]],
                 y="Load_KPlugLoadsPowerUsage", color="Status")

height = 1
for appliance, df in appliance_dict.items():
    for index in df.index:
        x = df.loc[index, "Timestamp"].timestamp() * 1000 - (2 + int(str(df.loc[index, "Timestamp"]).split("-")[-1][
                                                                         1])) * 3_600_000
        fig.add_vline(x=x,
                      line_width=3,
                      line_dash="dash",
                      annotation_text=df.loc[index, "Appliance"],
                      line_color=df.loc[index, "Color"],
                      annotation_y=height)
    height -= 0.05

fig.update_layout(
    xaxis={'type': 'date', 'tickformat': '%Y-%m-%d %H:%M:%S'}, )

fig.update_layout(
    xaxis_tickformatstops=[
        dict(dtickrange=[None, 60e6], value="%H:%M\n%d %b %y"),
        dict(dtickrange=[60e6, 10e10], value="%d %b\n%Y"),
        dict(dtickrange=[10e10, 10e100], value="%Y-%m-%d")])

fig.show()

In [48]:
power = year1_minutes_meta.loc[(year1_minutes_meta['Parameter'] == "Power_Electrical") & (year1_minutes_meta["max_value"] != 0)][
    "Unnamed: 0"].tolist()
columns = [col for col in power if col in year1_minutes.columns]
# columns = [year1_minutes.columns.get_loc(col) for col in power if col in year1_minutes.columns]

result_matrix = pc.fit_transform(X=year1_minutes.loc[non_outliers[0], columns].fillna(0))  #  [non_outliers][:]
result_df = pd.DataFrame(data=result_matrix,
                         columns=[f"pc{i + 1}" for i in range(result_matrix.shape[1])])
result_df.head()

,pc1,pc2,pc3,pc4,pc5
0,-1607.126248,-57.088719,-150.264529,389.993240,-154.826785
1,-1607.147954,-56.084605,-149.434309,386.465597,-154.884649
2,-1607.908214,-29.971395,-148.613054,383.430183,-154.983458
3,-1607.686843,-39.376328,-149.446151,386.639971,-154.953830
4,-1607.708665,-40.476733,-150.224959,389.670627,-154.975017


In [59]:
fig = px.scatter(result_df.loc[:1000], x="pc1", y="pc2")
fig.show()

In [60]:
fig = px.scatter(result_df.loc[:1000], x="pc1", y="pc3")
fig.show()

In [61]:
fig = px.scatter(result_df.loc[:1000], x="pc2", y="pc3")
fig.show()

In [80]:
fig = px.scatter_3d(result_df.loc[:10000], x="pc1", y="pc2", z="pc3")
fig.update_traces(marker=dict(size=2,
                              # line=dict(width=2)
                              ),
                  # selector=dict(mode='markers')
                  )
fig.write_html(Locator.get_onedrive_path("data/plots/3d_scatter_pca.html"))
fig.show()